# Analysis of one layer of 3DXRD data
This notebook illustrates how to run 3DXRD analysis on one slice with the help of DDDXRD and Fable.

In [14]:
import dddxrd.peaksearch.median_image as mi
import dddxrd.peaksearch.peaksearching as ps

import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np

## Peaksearching
The first step in the analysis is to search for diffraction peaks in all images. The peaks are identified based on the intensity values in the images by thresholding using one or more threshold values. The peaksearching is done using the peaksearch.py script from ImageD11. We use a .yaml file to pass parameters to the script. The result will be a bunch of *.flt and *.spt files (one for each threshold level) containing information about all identified peaks.

### Computing a background image
Before we can start looking for peaks we need to subtract the background. Typically there are a bunch of dark images in the dataset, so lets take the median of those and use as our background. The parameters for this are also set in a .yaml file.

In [15]:
# path to .yaml file containing parameter for median image
yaml_path = 'dddxrd/tests/median_image.yaml'
mimg = mi.median_image(yaml_path)
plt.figure()
plt.imshow(mimg,cmap=plt.cm.gray,vmin=np.mean(mimg)-2*np.std(mimg),vmax=np.mean(mimg)+2*np.std(mimg))

Looking for images in: dddxrd/tests
Found 2 images
Saved median image in /Users/al8720/code/DDDXRD/dddxrd/tests/median_dark.tif


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Finding diffraction spots
We are now ready to look for diffraction spots. The peaksearch will be distributed on all available cpus, so it shouldn't take too long. The will be a lot of printout and it will probably raise an exception but it seems to work nonetheless.

In [16]:
# path to .yaml file containing parameters for peaksearching
yaml_path = 'dddxrd/tests/peaksearch.yaml'
# run the peaksearch
ps.run_peaksearch(yaml_path)

Running peaksearch with the following command:
peaksearch.py -n dddxrd/tests/load_0N_ -F .tif -f 1 -l 2 -o dddxrd/tests/peaks/peaks -d dddxrd/tests/median_dark.tif -p Y --ndigits 5 -S 0.250 -T 0.125 -t 20000 -t 10000 -t 5000 --OmegaOverride 


### Merging the different thresholds
We can now merge the spots from the different threshold levels. This works by selecting all peaks from the highest threshold and adding peaks from the lower thresholds if they are not already present. For the peaks found in both levels, the lower threshold version is discared if the centre of mass position is not within pixel_tol pixels from the high threshold peak. We want to do this step to separate peaks that are close to eachother on the detector. This step will create a new .flt file containing the merged peaks.

In [17]:
yaml_path = 'dddxrd/tests/peaksearch.yaml'
ps.merge_peaks(yaml_path)

Merging flt files matching dddxrd/tests/peaks/peaks
merge_flt.py dddxrd/tests/junk dddxrd/tests/peaks/peaks dddxrd/tests/peaks/merged_peaks.flt 10 20000 10000 5000 
